# Required libraries and data

In [1]:
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

# Load the dataset
data = pd.read_excel("outpute.xlsx", sheet_name="Sheet1")

# Preprocess the text
preprocessed_text = data["Cleaned Text"].apply(lambda x: simple_preprocess(x))

# Train Word2Vec embeddings
model = Word2Vec(sentences=preprocessed_text, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Save the embeddings
model.save("word2vec_model.bin")


# Word2vec embeddings and cnn model

In [13]:


# Load the Word2Vec embeddings
word2vec_model = Word2Vec.load("word2vec_model.bin")

# Load the dataset
data = pd.read_excel("outpute.xlsx", sheet_name="Sheet1")

# Preprocess the text
preprocessed_text = data["Cleaned Text"].apply(lambda x: simple_preprocess(x))

# Convert text to numerical sequences
sequences = []
for text in preprocessed_text:
    sequence = [word2vec_model.wv.key_to_index[word] for word in text if word in word2vec_model.wv.key_to_index]
    sequences.append(sequence)

# Determine the maximum sequence length
max_length = max(len(seq) for seq in sequences)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Prepare labels
labels = (data["task_1"] == "HOF").astype(int)

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(len(padded_sequences) * split_ratio)

x_train, x_test = padded_sequences[:split_index], padded_sequences[split_index:]
y_train, y_test = labels[:split_index], labels[split_index:]

# Define the CNN model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(word2vec_model.wv.key_to_index), output_dim=embedding_dim, weights=[word2vec_model.wv.vectors], trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 16

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
predictions = model.predict(x_test)

# Classify the predictions as HOF or NOT based on a threshold
threshold = 0.5
classified_predictions = [1 if pred >= threshold else 0 for pred in predictions]

# Calculate classification scores
classification_scores = classification_report(y_test, classified_predictions)
print(classification_scores)


Epoch 1/10
234/234 [==============================] - 3s 7ms/step - loss: 0.6864 - accuracy: 0.5707
Epoch 2/10
234/234 [==============================] - 2s 7ms/step - loss: 0.6817 - accuracy: 0.5807
Epoch 3/10
234/234 [==============================] - 2s 7ms/step - loss: 0.6555 - accuracy: 0.6112
Epoch 4/10
234/234 [==============================] - 2s 7ms/step - loss: 0.6504 - accuracy: 0.6184
Epoch 5/10
234/234 [==============================] - 2s 7ms/step - loss: 0.6474 - accuracy: 0.6316
Epoch 6/10
234/234 [==============================] - 2s 7ms/step - loss: 0.6462 - accuracy: 0.6254
Epoch 7/10
234/234 [==============================] - 2s 7ms/step - loss: 0.6417 - accuracy: 0.6434
Epoch 8/10
234/234 [==============================] - 2s 7ms/step - loss: 0.6393 - accuracy: 0.6369
Epoch 9/10
234/234 [==============================] - 2s 7ms/step - loss: 0.6419 - accuracy: 0.6412
Epoch 10/10
30/30 [==============================] - 0s 5ms/step - loss: 0.7524 - accuracy: 0.5005
T

# LSTM Model

In [2]:

# Load the Word2Vec embeddings
word2vec_model = Word2Vec.load("word2vec_model.bin")

# Load the dataset
data = pd.read_excel("outpute.xlsx", sheet_name="Sheet1")

# Preprocess the text
preprocessed_text = data["Cleaned Text"].apply(lambda x: simple_preprocess(x))

# Convert text to numerical sequences
sequences = []
for text in preprocessed_text:
    sequence = [word2vec_model.wv.key_to_index[word] for word in text if word in word2vec_model.wv.key_to_index]
    sequences.append(sequence)

# Determine the maximum sequence length
max_length = max(len(seq) for seq in sequences)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Prepare labels
labels = (data["task_1"] == "HOF").astype(int)

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(len(padded_sequences) * split_ratio)

x_train, x_test = padded_sequences[:split_index], padded_sequences[split_index:]
y_train, y_test = labels[:split_index], labels[split_index:]

# Define the LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(word2vec_model.wv.key_to_index), output_dim=embedding_dim, weights=[word2vec_model.wv.vectors], trainable=False))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 16

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
predictions = model.predict(x_test)

# Classify the predictions as HOF or NOT based on a threshold
threshold = 0.5
classified_predictions = [1 if pred >= threshold else 0 for pred in predictions]

# Calculate classification scores
classification_scores = classification_report(y_test, classified_predictions)
print(classification_scores)


Epoch 1/10
234/234 [==============================] - 8s 24ms/step - loss: 0.6878 - accuracy: 0.5528
Epoch 2/10
234/234 [==============================] - 5s 23ms/step - loss: 0.6560 - accuracy: 0.6069
Epoch 3/10
234/234 [==============================] - 5s 23ms/step - loss: 0.6388 - accuracy: 0.6313
Epoch 4/10
234/234 [==============================] - 5s 23ms/step - loss: 0.6299 - accuracy: 0.6546
Epoch 5/10
234/234 [==============================] - 5s 23ms/step - loss: 0.6260 - accuracy: 0.6509
Epoch 6/10
234/234 [==============================] - 5s 23ms/step - loss: 0.6218 - accuracy: 0.6533
Epoch 7/10
234/234 [==============================] - 6s 24ms/step - loss: 0.6142 - accuracy: 0.6675
Epoch 8/10
234/234 [==============================] - 5s 23ms/step - loss: 0.6153 - accuracy: 0.6635
Epoch 9/10
234/234 [==============================] - 6s 24ms/step - loss: 0.6117 - accuracy: 0.6699
Epoch 10/10
30/30 [==============================] - 1s 12ms/step - loss: 0.6999 - accuracy

# Bi-LSTM 

In [3]:

# Load the Word2Vec embeddings
word2vec_model = Word2Vec.load("word2vec_model.bin")

# Load the dataset
data = pd.read_excel("outpute.xlsx", sheet_name="Sheet1")

# Preprocess the text
preprocessed_text = data["Cleaned Text"].apply(lambda x: simple_preprocess(x))

# Convert text to numerical sequences
sequences = []
for text in preprocessed_text:
    sequence = [word2vec_model.wv.key_to_index[word] for word in text if word in word2vec_model.wv.key_to_index]
    sequences.append(sequence)

# Determine the maximum sequence length
max_length = max(len(seq) for seq in sequences)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Prepare labels
labels = (data["task_1"] == "HOF").astype(int)

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(len(padded_sequences) * split_ratio)

x_train, x_test = padded_sequences[:split_index], padded_sequences[split_index:]
y_train, y_test = labels[:split_index], labels[split_index:]

# Define the Bi-LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(word2vec_model.wv.key_to_index), output_dim=embedding_dim, weights=[word2vec_model.wv.vectors], trainable=False))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 16

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
predictions = model.predict(x_test)

# Classify the predictions as HOF or NOT based on a threshold
threshold = 0.5
classified_predictions = [1 if pred >= threshold else 0 for pred in predictions]

# Calculate classification scores
classification_scores = classification_report(y_test, classified_predictions)
print(classification_scores)


Epoch 1/10
234/234 [==============================] - 8s 26ms/step - loss: 0.6783 - accuracy: 0.5702
Epoch 2/10
234/234 [==============================] - 7s 29ms/step - loss: 0.6445 - accuracy: 0.6270
Epoch 3/10
234/234 [==============================] - 6s 27ms/step - loss: 0.6348 - accuracy: 0.6407
Epoch 4/10
234/234 [==============================] - 7s 29ms/step - loss: 0.6317 - accuracy: 0.6565
Epoch 5/10
234/234 [==============================] - 7s 29ms/step - loss: 0.6326 - accuracy: 0.6404
Epoch 6/10
234/234 [==============================] - 7s 28ms/step - loss: 0.6296 - accuracy: 0.6474
Epoch 7/10
234/234 [==============================] - 7s 29ms/step - loss: 0.6201 - accuracy: 0.6586
Epoch 8/10
234/234 [==============================] - 7s 29ms/step - loss: 0.6159 - accuracy: 0.6645
Epoch 9/10
234/234 [==============================] - 6s 27ms/step - loss: 0.6100 - accuracy: 0.6685
Epoch 10/10
30/30 [==============================] - 1s 11ms/step - loss: 0.7095 - accuracy